In [ ]:
!python -m pip install matplotlib numpy

In [ ]:
from math import cos, erfc, pow, sin, sqrt
from sys import float_info

import numpy as np
from numpy.linalg import norm, solve

def lsq_spline(x: np.ndarray, y: np.ndarray, knots: np.ndarray,
        order: int = 3, robust: bool = False, delta: float = 1.0,
        epsilon: float = 1E-7, max_iter: int = 100) -> np.ndarray:
    if (not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)
            or not isinstance(knots, np.ndarray) or not isinstance(order, int)
            or not isinstance(robust, bool) or not isinstance(delta, float)
            or not isinstance(epsilon, float) or not isinstance(max_iter, int)):
        raise TypeError()
    if (len(x.shape) != 1 or len(x) < 2 or np.any(np.diff(x) <= 0)
            or x.shape != y.shape or len(knots.shape) != 1 or len(knots) < 2
            or np.any(np.diff(knots) <= 0) # TODO x[0] >= knots[0] or knots[-1] >= x[-1]
            or order < 1 or delta < float_info.epsilon
            or epsilon < float_info.epsilon or max_iter < 1):
        raise ValueError()

    n, m = len(x), len(knots)
    x_upper = np.zeros((n, m+nodes_num))
    for j in range(order + 1):
        for i in range(m):
            if (x[j] > knots[i]):
                Phi[j, i] += pow(x[i], j)
    for j in range(n):
        for i in range(m):
            if (x[j] > knots[i]):
                Phi[j, i] += pow(x[j] - knots[i], order)


    x_upper = np.vander(x, m, increasing=False)
    left, right = x_upper.T @ x_upper, x_upper.T @ y
    beta = solve(left, right)
    if not robust:
        return beta

    for _ in range(max_iter):
        x_upper_beta = x_upper @ beta
        diff = x_upper_beta - y
        cond = np.abs(diff) < delta
        y = np.where(cond, y, x_upper_beta - np.sign(diff) * delta)
        right = x_upper.T @ y
        next_beta = solve(left, right)
        error = norm(next_beta - beta, ord=2)
        beta, next_beta = next_beta, beta
        if (error < epsilon):
            break
    return beta

In [ ]:
def f(x: float) -> float:
    return x**2 * 5.0 / 7.0 - 5.0 * x + 5.0 + 11.0 * sin(x * 7.0 / 3.0)

start, stop = -7.0, 17.0
x = np.linspace(start, stop, 2 ** 8 + 1)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import choice, normal

n = len(x)
y_noise = y + normal(loc=0.0, scale=1.0, size=n)
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
p = erfc(sqrt(2))
typical = normal(loc=0.0, scale=1.0, size=n)
outlier = normal(loc=0.0, scale=100.0, size=n)
cond = choice([False, True], size=n, p=[p, 1.0 - p])
y_noise = y + np.where(cond, typical, outlier)
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
poly = ordinary_least_squares(x, y_noise, order=2)
polyval = np.polyval(poly, x)
print(poly)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, polyval, color='blue', lw=2, label='polyval')
plt.legend()
plt.show()

In [ ]:
poly = ordinary_least_squares(x, y_noise, order=2, robust=True, delta = 5.0)
polyval = np.polyval(poly, x)
print(poly)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, polyval, color='blue', lw=2, label='polyval')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline

start, end = -20, 20
x = np.linspace(start, end, 2 ** 10 + 1)
y = x + 5.0 * np.sin(x / 2.0) + 11.0 * np.sin(x / 5.0) + 0.1 * np.random.randn(len(x))
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, 'ro', ms=5)

spl = UnivariateSpline(x, y, k=4, s=500.0)
xs = np.linspace(start, end, len(x) * 2)
plt.plot(xs, spl(xs), 'g', lw=3)
spl.get_knots()